## Elsevier API with generated key with university account:

In [30]:
import requests
import csv

api_key = '02d0c4174287f162c45c7853a4ccd18e'


# Define the API endpoint and query parameters
api_url = 'https://api.elsevier.com/content/search/sciencedirect'
query_params = {
    # 'query': 'pub-date(2020-2022) AND random(1000)',
    'apiKey': api_key,
    'view': 'COMPLETE',
    'content': 'metadata',
    'start': 0,  # Start with the first result
    'count': 1000,  # Retrieve 1000 results
}

# Make the API request
response = requests.get(api_url, params=query_params) 

# Check the response status
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    publications = []
    for entry in data['search-results']['entry']:
        title = entry['dc:title']
        year = entry['prism:coverDisplayDate']
        abstract = entry['dc:description']
        citations = entry.get('citedby-count', 'N/A')
        publications.append([title, year, abstract, citations])

    # Save the data to a CSV file
    with open('publications.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write the header row
        csv_writer.writerow(['Title', 'Year', 'Abstract', 'Citations'])
        # Write the publication data
        csv_writer.writerows(publications)

    print("Data saved to 'publications.csv'")
else:
    print(f"API request failed with status code {response.status_code}")


API request failed with status code 405


Doesn't work:
"The 405 error happens when a client attempts to use an HTTP method that is not allowed by the server. For example, this issue can occur when a user tries to use a method such as PUT or DELETE on a resource that only allows GET or POST requests."

Not sure how to resolve this


Also 'query': 'pub-date(2020-2022) AND random(1000) does not work (that's why I commented it out), it gives a 401 error:
"The HyperText Transfer Protocol (HTTP) 401 Unauthorized response status code indicates that the client request has not been completed because it lacks valid authentication credentials for the requested resource."

